In [42]:
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import os
from pathlib import Path
home = str(Path.home())

In [43]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)
    kwargs = {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 100, 'C': 0.01}
    clf = LogisticRegression(n_jobs=-1, **kwargs)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [44]:
def predictions(clf, X_test, scaler=None):
    if not scaler == None:
        X_test   = scaler.transform(X_test)
    y_hat    = clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    return y_hat, y_hat_pr

In [45]:
def RF(X_train, y_train, X_test, y_test):
    kwargs = {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 60, 'bootstrap': False}  

    clf = RandomForestClassifier(n_jobs=-1, **kwargs)
    clf.fit(X_train, y_train)
    y_hat, y_hat_pr = predictions(clf, X_test)
    
    return y_hat, y_hat_pr

In [46]:
factor = 0.8
nr_mean = 3
mean_tables = []
nr_samples=2000

attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# datasets = ['cifar10']
datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
detectors = ['multilid']
clfs = ['lr', 'rf']

results = {}

for it in range(nr_mean):
    random_state = [21, 30, 65][it] # random.randint(100)
    print("random state: ", random_state)
    final_table = np.zeros((len(attacks), 2))
    base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
    
    results[it] = {}
    
    for dataset in datasets:
        
        if dataset == 'imagenet':
            k=30
        else:
            k=20
        
        for model in models:
            
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
                
            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}
                

            
            for detector in detectors:
                
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]
                 
                for rows in range(len(nor)):
                    X_nor = nor[rows]
                    X_adv = adv[rows]
                    if len(X_nor.shape) > 2:
                        X_nor = nor[rows].reshape((nor[rows].shape[0], -1))
                        X_adv = adv[rows].reshape((adv[rows].shape[0], -1))

                    y_nor = np.zeros(X_nor.shape[0]).astype('int')
                    y_adv = np.ones(X_nor.shape[0]).astype('int')

                    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(X_nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
                    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(X_adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

                    X_train = np.concatenate((x_train_n, x_train_a))
                    y_train = np.concatenate((y_train_n, y_train_a))

                    X_test = np.concatenate((x_test_n, x_test_a))
                    y_test = np.concatenate((y_test_n, y_test_a))
                    for clf in clfs:
                        if clf == 'lr':
                            y_hat, y_hat_pr = LR(X_train, y_train, X_test, y_test)
                        elif  clf == 'rf':
                            y_hat, y_hat_pr = RF(X_train, y_train, X_test, y_test)

                        auc = round(100*roc_auc_score(y_test, y_hat_pr), 2)
                        f1 =  round(100*f1_score(y_test, y_hat), 2)
                        
                        if not clf in results[it][dataset][model]:
                            results[it][dataset][model][clf] = {}
                        if not attacks[rows] in results[it][dataset][model][clf]:
                            results[it][dataset][model][clf][attacks[rows]] = {}
                        if not attacks[rows] in results[it][dataset][model][clf]:
                            results[it][dataset][model][clf][attacks[rows]] = {}

                        # if not 'auc' in results[it][dataset][model][clf][attacks[rows]]:
                        #     results[it][dataset][model][clf][attacks[rows]]['auc'] = {}
                        # if not 'f1' in results[it][dataset][model][clf][attacks[rows]]:
                        #     results[it][dataset][model][clf][attacks[rows]]['f1' ] = {}

                        results[it][dataset][model][clf][attacks[rows]]['auc'] = auc
                        results[it][dataset][model][clf][attacks[rows]]['f1'] = f1                    

                    # print(attacks[rows].upper(), '&' , auc, '&', f1)

random state:  21


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.c

random state:  30


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.c

random state:  65


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/lorenzp/.c

# LR

In [47]:
lid_list_mean = []
for it_mean in range(nr_mean):
    lid_list = []
    for dataset in ['cifar10', 'cifar100', 'imagenet']:
        for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            tmp = np.zeros((len(attacks), 2))
            for it, val in enumerate(results[it_mean][dataset][model]['lr'].items()):
                # print(it, val)
                tmp[it, 0] = val[1]['auc'].copy()
                tmp[it, 1] = val[1]['f1'].copy()
            lid_list.append(tmp.copy())
    
    lid_list_mean.append(np.stack(lid_list))
lid_list_mean_res = np.stack(lid_list_mean)
lid_mean = np.mean(lid_list_mean_res, axis=0)
lid_var = np.var(lid_list_mean_res, axis=0)

In [48]:
lid_mean.shape[0]

5

In [49]:
latex_table = ""
for j in range(lid_mean.shape[1]):
    latex_table += "\\textbf{"+f"{attacks[j].upper()}"+"} & "
    for i in range(lid_mean.shape[0]):
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
    latex_table = latex_table[:-2] + "\\\\ \n"

print(latex_table)

\textbf{FGSM} & $97.64 \pm 0.16$ & $91.58 \pm 1.40$ & $93.24 \pm 0.05$ & $86.39 \pm 1.23$ & $99.10 \pm 0.05$ & $95.06 \pm 0.33$ & $85.32 \pm 0.02$ & $78.41 \pm 0.84$ & $80.65 \pm 3.30$ & $73.15 \pm 3.08$ \\ 
\textbf{BIM} & $95.44 \pm 0.36$ & $90.40 \pm 0.28$ & $93.59 \pm 0.88$ & $87.02 \pm 1.27$ & $98.29 \pm 0.04$ & $92.84 \pm 0.14$ & $84.76 \pm 0.72$ & $77.76 \pm 3.38$ & $96.78 \pm 0.02$ & $90.54 \pm 0.32$ \\ 
\textbf{PGD} & $96.60 \pm 0.36$ & $91.87 \pm 1.34$ & $90.91 \pm 0.95$ & $82.81 \pm 0.56$ & $98.89 \pm 0.00$ & $95.26 \pm 0.35$ & $89.10 \pm 0.18$ & $82.24 \pm 0.09$ & $97.44 \pm 0.06$ & $92.28 \pm 0.01$ \\ 
\textbf{AA} & $99.00 \pm 0.06$ & $95.74 \pm 0.09$ & $98.25 \pm 0.24$ & $93.70 \pm 0.87$ & $99.89 \pm 0.00$ & $98.27 \pm 0.04$ & $91.27 \pm 0.71$ & $83.66 \pm 0.74$ & $99.95 \pm 0.00$ & $98.76 \pm 0.04$ \\ 
\textbf{DF} & $97.93 \pm 0.03$ & $94.06 \pm 0.04$ & $89.54 \pm 1.02$ & $84.34 \pm 1.02$ & $75.99 \pm 0.73$ & $70.77 \pm 1.79$ & $70.76 \pm 4.48$ & $67.15 \pm 2.55$ & $54.49

# multiLID

In [50]:
lid_list_mean = []
for it_mean in range(nr_mean):
    lid_list = []
    for dataset in ['cifar10', 'cifar100', 'imagenet']:
        for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            tmp = np.zeros((len(attacks), 2))
            for it, val in enumerate(results[it_mean][dataset][model]['rf'].items()):
                # print(it, val)
                tmp[it, 0] = val[1]['auc'].copy()
                tmp[it, 1] = val[1]['f1'].copy()
            lid_list.append(tmp.copy())
    
    lid_list_mean.append(np.stack(lid_list))
lid_list_mean_res = np.stack(lid_list_mean)
lid_mean = np.mean(lid_list_mean_res, axis=0)
lid_var = np.var(lid_list_mean_res, axis=0)

In [51]:
latex_table = ""
for j in range(lid_mean.shape[1]):
    latex_table += "\\textbf{"+f"{attacks[j].upper()}"+"} & "
    for i in range(lid_mean.shape[0]):
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
    latex_table = latex_table[:-2] + "\\\\\n"

# Print the LaTeX table
print(latex_table)

\textbf{FGSM} & $96.84 \pm 0.14$ & $90.90 \pm 0.70$ & $91.04 \pm 0.24$ & $82.97 \pm 1.32$ & $98.47 \pm 0.31$ & $94.31 \pm 2.28$ & $82.56 \pm 0.77$ & $76.61 \pm 1.21$ & $79.52 \pm 2.09$ & $72.85 \pm 0.78$ \\
\textbf{BIM} & $96.11 \pm 0.28$ & $89.61 \pm 1.77$ & $94.55 \pm 0.33$ & $88.37 \pm 0.99$ & $97.74 \pm 0.12$ & $91.48 \pm 0.14$ & $83.14 \pm 0.29$ & $74.94 \pm 0.13$ & $94.33 \pm 0.26$ & $87.43 \pm 2.04$ \\
\textbf{PGD} & $97.50 \pm 0.11$ & $92.39 \pm 0.13$ & $92.18 \pm 1.07$ & $84.00 \pm 2.62$ & $98.77 \pm 0.06$ & $94.82 \pm 1.08$ & $88.36 \pm 0.09$ & $81.40 \pm 0.50$ & $96.55 \pm 0.19$ & $90.79 \pm 0.21$ \\
\textbf{AA} & $99.54 \pm 0.01$ & $96.96 \pm 0.05$ & $98.61 \pm 0.02$ & $94.74 \pm 1.41$ & $99.87 \pm 0.00$ & $98.20 \pm 0.02$ & $91.25 \pm 0.36$ & $83.76 \pm 0.87$ & $99.88 \pm 0.01$ & $99.08 \pm 0.01$ \\
\textbf{DF} & $97.47 \pm 0.21$ & $94.20 \pm 0.12$ & $89.04 \pm 1.61$ & $84.43 \pm 1.11$ & $74.85 \pm 0.58$ & $70.11 \pm 1.71$ & $73.44 \pm 0.66$ & $69.09 \pm 0.96$ & $53.41 \pm